# Simulation Chaotic Magnetic Pendulum

This code calculates the positions and velocities of the magnetic pendulum using the positions of the arragment of magnets on the table and the initial conditions of this magnetic pendulum. The solution was obtained using the RK4 method. Besides, this code also analysis the data obtaines throw animations of the movement, phase portraits, Liapunov exponents, and basins of atraction.

The structure of the code follows the nex points:
1. [Create the code to find and animate the trajectories](#section1)
    1. Class of magnetic pendulum
    2. Animation
    
2. [Run the code and show results](#section2)
    1. Normal run
    2. Load data of other runs
    3. Plot potential and trajectory
    4. Aditional isues
    
3. [Analyse the data](#section3)
    1. Phase Portrait
    2. Difference of paths and Liapunov exponent
    3. Lyapunov Exponent Analysis

In [1]:
from matplotlib import cm  #Color maps
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time

## 1. Create the code to find and animate the trajectories  <a id='section1'></a>

### A. Class of the magnetic pendulum

In [2]:
class pendulum:
    def __init__(self):
        # System dynamics parameters
        self.R = 0          #friction constant   
        self.l = 0.54       #Pendulum length
        self.d = 0.03       #vertical distance between pendulum and magnets
        self.h = 0.001       #time lapse between each step
        self.N = 60000       #total number of steps
        self.m = 0.036226   #Mass of the pendulum magnet
        self.mu_P_magn = 2  #Magnitude of the magnetic dipole of the pendulum magnet
        
        # Other parameters
        self.sec = 1/self.h #Number of steps that waste one second.
        self.Table_SIZE = 0.3
        self.NMAGNETS = 9
        self.lim = self.Table_SIZE/2 + 0.04         #Limit of the plots
        self.lim_vs = (self.Table_SIZE + 0.05)/2   #Limit plot of vector space
        
        # Initial velocities
        self.vx = 0
        self.vy = 0
        
        # Set initial position randomly    
        lm = self.Table_SIZE
        self.x = np.random.rand()*lm -lm/2
        self.y = np.random.rand()*lm -lm/2
              
        # Create table
        self.Mx0 = [0.14233062820745887, 0.02944837371034076, -0.05426532834852549, 
              -0.06032737886107843, -0.08297754374954086, 0.11916400502656863, 
               0.04521514008679134, -0.0233, 0.014947564108854505]
        self.My0 = [0.06427852819354979,  0.021412071373469233, 0.025695474931522272,
             -0.125554841803107, -0.068025984035971, -0.048248682583613894,
             -0.10589038572208646, -0.0337, 0.0809038924904392]
        self.Mx0 = np.array(self.Mx0)
        self.My0 = np.array(self.My0)

        rho = np.sqrt(self.Mx0**2+self.My0**2)
        self.Mx = 0.05*self.Mx0/max(rho)
        self.My = 0.05*self.My0/max(rho)
        
        self.NMAGNETS = len(self.Mx)
        
        # Create dipoles
        self.mu = np.array([2.5]*len(self.Mx))
        self.mu_magn = np.abs(np.array(self.mu))
        
        self.mu_dir = [np.array([0,0,-1])*np.sign(self.mu[i]) for i in range(self.NMAGNETS)] #Direction magnetic dipoles of table magnets
        self.mu_dir = np.array(self.mu_dir)
        self.mu_NUMS=len(self.mu_magn)
        
        #Finding S values
        self.S = np.copy(self.mu_magn)
        self.S = 3*(4*np.pi*10**-7 * self.mu_P_magn) / (4*np.pi) * self.S
    
        self.sign = [np.sign(self.mu[i]) for i in range(self.NMAGNETS)] #If sign is positive there is atraction; if negative, repulsion
    
    def set_hN(self,h,N):
        self.h = h
        self.N = N
        self.sec = 1/self.h #Number of steps that waste one second.
        
    def set_r(self,r):
        self.Mx = r*self.Mx0/max(rho)
        self.My = r*self.My0/max(rho)
    
    def set_CI(self, CI):
        self.x = CI[0]
        self.y = CI[1]
        self.vx = CI[2]
        self.vy = CI[3]
    
    def set_lim(self, lim):
        self.lim = lim
    
    def set_positions(self,CI, Mx, My):
        self.x = CI[0]
        self.y = CI[1]
        self.vx = CI[2]
        self.vy = CI[3]
        
        self.Mx = Mx
        self.My = My
        self.NMAGNETS = Mx.size
        
    def set_code_parameters(self, m, l, d):
        self.m = m     #Mass of the pendulum
        self.l = l     #lenght of the pendulum
        self.d = d     #Distance between table and pendulum equilibrium position
    
    def set_magnetic_dipoles(self, mu_P, mu):
        self.mu_P_magn = np.abs(mu_P)  #Magnitude of the magnetic dipole
        self.mu_magn = np.abs(np.array(mu))
        self.mu_dir = [np.array([0,0,-1])*np.sign(mu[i]) for i in range(self.NMAGNETS)] #Direction magnetic dipoles of table magnets
        
        self.mu_dir = np.array(self.mu_dir)
        self.mu_NUMS=len(self.mu_magn)
        
        #Finding S values
        self.S = np.copy(self.mu_magn)
        self.S = 3*(4*np.pi*10**-7 * self.mu_P_magn) / (4*np.pi) * self.S
    
        self.sign = [np.sign(self.mu[i]) for i in range(self.NMAGNETS)] #If sign is positive there is atraction; if negative, repulsion
    
    def input_path(self, P):
        self.X = P[0]
        self.Y = P[1]
        self.Vx = P[2]
        self.Vy = P[3]
    
    def get_self(self):
        return self
    
    def find_path(self):
        # Finding the next steps for each interval
        self.X = [self.x]    #X positions
        self.Y = [self.y]    #Y positions
        self.Vx = [self.vx]  #X velocities
        self.Vy = [self.vy]  #Y velocities
        self.FB = []         #Arrays of the net magnetic forces
        self.B = []         #Arrays of the net magnetic fields
        
        s=self.sec #Number of steps that waste one second.
        A = int(1.2*s) #check point 1
        B = int(0.6*s) #check point 2
        C = int(0.2*s) #check point 3
        Rc = (0.01)**2 #squared Critical distance to stop the calculations. The magnitude is in meters^2
        
        #Finfing path
        for t in range(self.N):
            next_val=self.next_value(self.X[t],self.Y[t],self.Vx[t],self.Vy[t])
            
            self.FB.append(self.FF2)
            self.B.append(self.BB2)
            self.X.append(next_val[0])
            self.Y.append(next_val[1])
            self.Vx.append(next_val[2])
            self.Vy.append(next_val[3])
            
            #if t>5*s:  #This conditional returns true when the time is higher than 5 seconds
            #    if ((self.X[t-A]-self.X[t])**2 + (self.Y[t-A]-self.Y[t])**2 < Rc):
            #        if ((self.X[t-B]-self.X[t])**2 + (self.Y[t-B]-self.Y[t])**2 < Rc):
            #            if ((self.X[t-C]-self.X[t])**2 + (self.Y[t-C]-self.Y[t])**2 < Rc):
            #                break
            
        # Find the last net magnetic force and net magnetic field
        aux = self.next_value(self.X[-1],self.Y[-1],self.Vx[-1],self.Vy[-1])
        self.FB.append(self.FF2)
        self.B.append(self.BB2)
        
        # Converting list to arrays
        self.X = np.array(self.X)
        self.Y = np.array(self.Y)
        self.Vx = np.array(self.Vy)
        self.Vy = np.array(self.Vx)
        
        return self.X, self.Y, self.Vx, self.Vy
    
    def plot_path(self):
        # Plot trajectory
        plt.plot(self.X,self.Y,'-b',linewidth=0.5)
        
        # Initial and final position
        plt.plot(self.X[0],self.Y[0],'Dg',label=f'Initial position ({np.round(self.X[0],3)},{np.round(self.Y[0],3)})', markersize=5)
        plt.plot(self.X[-1],self.Y[-1],'Dc',label='Final position', markersize=5)
        
        # Put legends, title, labales, and ticks
        plt.legend(fontsize=12)
        plt.title('Trajectory of the magnet',fontsize=15)
        plt.xlabel('x-axis [m]',fontsize=12)
        plt.ylabel('y-axis [m]',fontsize=12)
        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)
        
    def plot_table(self):
        
        # Plot magnets
        plt.figure(figsize=(6,6))
        for i in range(self.NMAGNETS):
            plt.plot(self.Mx[i],self.My[i],'or')
        plt.plot(self.Mx[0],self.My[0],'or',label='Magnets')
        
        # Plot Table
        self.ap = self.Table_SIZE/2 +0.01
        plt.plot([-self.ap,-self.ap,self.ap,self.ap,-self.ap],[-self.ap,self.ap,self.ap,-self.ap,-self.ap],'-b',label='Table')
        
        # Put legends, title, labales, and limits
        plt.legend()
        plt.title('Positions of magnets')
        plt.xlabel('x-axis [m]')
        plt.ylabel('y-axis [m]')
        plt.xlim(-self.lim,self.lim)
        plt.ylim(-self.lim,self.lim) 

    def plot_vector_space(self,res=20,a=0):
        
        # Put limits of plot
        if a==0:
            a=self.lim_vs #Predefined value
        elif a > self.l:
            a = self.l/2 #Ensure that the points are ploted in the zone inside the spheric pendulum
        
        # Create data of vector tails
        t = np.linspace(-a,a,res)
        NUM = len(t)
        X, Y = np.meshgrid(t,t)
        Z = -np.sqrt(self.l**2 - X**2 - Y**2)+self.l+self.d
        
        # Finding net magnetic force
        Fx = np.zeros((NUM,NUM))
        Fy = np.zeros((NUM,NUM))
        Fz = np.zeros((NUM,NUM))
        
        # Unit vector of the magnetic dipole m1 (from Q to P)
        M1x = X/self.l                 #X component 
        M1y = Y/self.l                 #Y component 
        M1z = (Z-self.l-self.d)/self.l #Z component 
        
        # Iterating over table magnets
        for i in range(self.NMAGNETS):
            
            #Squared distance between magnet i and pendulum magnet
            R = (self.Mx[i]-X)**2 + (self.My[i]-Y)**2 + Z**2  
            
            #Table magnet constants
            m2 = self.mu_dir[i]
            
            #Unit vector from m2 (on table) to m1 (in pendulum)
            Ur_x = (X-self.Mx[i])/np.sqrt(R) #X component 
            Ur_y = (Y-self.My[i])/np.sqrt(R) #Y component 
            Ur_z = Z/np.sqrt(R)              #Z component 
            
            #Dot products
            M12 = M1x*m2[0] + M1y*m2[1] + M1z*m2[2]     #Dot products between m1 and m2
            M1r = M1x*Ur_x + M1y*Ur_y + M1z*Ur_z        #Dot products between m1 and ur
            M2r = m2[0]*Ur_x + m2[1]*Ur_y + m2[2]*Ur_z  #Dot products between m2 and ur
            
            #Sum contributions of all magnets
            Fx = Fx + self.S[i] / R**2 * ( (M12)*Ur_x + (M2r)*M1x + (M1r)*m2[0] - 5*(M1r)*(M2r)*Ur_x )
            Fy = Fy + self.S[i] / R**2 * ( (M12)*Ur_y + (M2r)*M1y + (M1r)*m2[1] - 5*(M1r)*(M2r)*Ur_y )
            Fz = Fz + self.S[i] / R**2 * ( (M12)*Ur_z + (M2r)*M1z + (M1r)*m2[0] - 5*(M1r)*(M2r)*Ur_z )
        
        # Magnitude of the projection of net magnetic force on plane xy
        F_xy_mag = np.sqrt(Fx**2 + Fy**2)
        
        # Finding tension Force
        
        # Unit vector of tension (from P to Q) 
        UT_x = -X/self.l                #X component 
        UT_y = -Y/self.l                #Y component 
        UT_z = (self.l+self.d-Z)/self.l #Z component 
        
        # Dot product
        FbUT = Fx*UT_x + Fy*UT_y + Fz*UT_z  #Dot product between net Fb and UT
        
        # Finding Tension
        Tx = (self.m*9.8*(self.l+self.d-Z)/self.l - FbUT)*UT_x
        Ty = (self.m*9.8*(self.l+self.d-Z)/self.l - FbUT)*UT_y
        Tz = (self.m*9.8*(self.l+self.d-Z)/self.l - FbUT)*UT_z
        
        # Magnitude of the projection of tension force on plane xy
        T_xy_mag = np.sqrt(Tx**2 + Ty**2)
        
        # Find force field (refusing friction force)
        Field_x = Fx+Tx
        Field_y = Fy+Ty
        Field_z = Fz+Tz
        
        # Magnitude of the projection of force field on plane xy
        Field_xy_mag = np.sqrt(Field_x**2 + Field_y**2)
        
        # Plotting vectors
        
        # Field Forces
        plt.quiver(X,Y,Field_x/Field_xy_mag, Field_y/Field_xy_mag, width=0.0025, angles='xy', scale_units='width', scale=36)
        
        #Magnetic net forces
        #plt.quiver(X,Y,Fx/F_xy_mag,Fy/F_xy_mag, linewidths=4, width=0.0025, angles='xy', scale_units='width', scale=39)                
        
        #Tensions
        #plt.quiver(X,Y,Tx/T_xy_mag, Ty/T_xy_mag, linewidths=4, width=0.0025, angles='xy', scale_units='width', scale=37)

    
    def plot_potential(self,res=20, a=0):
        
        # Put limits of plot
        if a==0:
            a = self.lim_vs #predifined value
        elif a > self.l:
            a = self.l/2 #Ensure that the points are ploted in the zone inside the spheric pendulum
        
        # Create data of the plot
        t = np.linspace(-a,a,res)
        NUM = len(t)
        X, Y = np.meshgrid(t,t)
        Z = -np.sqrt(self.l**2 - X**2 - Y**2)+self.l+self.d

        # Finding magnetic field B
        Bx = np.zeros((NUM,NUM))
        By = np.zeros((NUM,NUM))
        Bz = np.zeros((NUM,NUM))
        
        # Unit vector of the magnetic dipole of m1 (vector from Q to P)
        M1x = X/self.l                 #X component
        M1y = Y/self.l                 #Y component 
        M1z = (Z-self.l-self.d)/self.l #Z component 
        
        # Iterating over each table magnet
        for i in range(self.NMAGNETS):
            
            #Squared distance between magnet i and the pendulum magnet
            R = (self.Mx[i]-X)**2 + (self.My[i]-Y)**2 + Z**2  
            
            #Table magnet constants
            m2 = self.mu_dir[i]
            Cm = -(4*np.pi*10**-7 * self.mu_magn[i]) / (4*np.pi)
            
            #Unit vector from m2 (on table) to m1 (in pendulum)
            Ur_x = (X-self.Mx[i])/np.sqrt(R) #X component
            Ur_y = (Y-self.My[i])/np.sqrt(R) #Y component 
            Ur_z = Z/np.sqrt(R)              #Z component 
            
            #Dot products
            M2r = m2[0]*Ur_x + m2[1]*Ur_y + m2[2]*Ur_z  #Dot products between m2 and ur
            
            #Sum contributions of all magnets
            Bx = Bx + Cm/R**(3/2) * (m2[0] - 3*M2r*Ur_x)
            By = By + Cm/R**(3/2) * (m2[1] - 3*M2r*Ur_y)
            Bz = Bz + Cm/R**(3/2) * (m2[2] - 3*M2r*Ur_z)
        
        # Finding magnetic potential
        
        # Unit vector from Q to P (direction of m1)
        Um1_x = X/self.l                 #X component  
        Um1_y = Y/self.l                 #Y component 
        Um1_z = (Z-self.l-self.d)/self.l #Z component 
        
        # Pendulum magnet constants
        m1 = self.mu_P_magn
        
        # Magnetic potential
        Um = -m1 * (Um1_x*Bx + Um1_y*By + Um1_z*Bz)
        
        # Finding gravitational potential
        Ug = self.m*9.8*(Z-self.d)
        
        # Plot potential
        fig = plt.figure(figsize=(5,4))
        ax = plt.axes(projection='3d')
        surf = ax.plot_surface(X,Y,Ug+Um, cmap=cm.jet, edgecolor='none')
        fig.colorbar(surf,shrink=0.5, aspect=8, pad = 0.15)
        
        plt.title(f'Potential Energy', fontsize=15)
        plt.xlabel(r'X [m]',fontsize=12)
        plt.ylabel(r'Y [m]',fontsize=12)
        ax.set_zlabel(r'Ug+Um [J]',fontsize=12)
        plt.xticks(fontsize=9)
        plt.yticks(fontsize=9)
        for t in ax.zaxis.get_major_ticks(): t.label.set_fontsize(9)
            
    def Fk(self,x,y,vx,vy):
        """
        Calculates the velocity and acelerations based on the initial conditions of the magnets 
        @params
        mx, my:
            arrays of doubles that contains the position of the magnets
        x,y,vx,vy:
            doubles of the position and velocity of the pendulum magnet

        @returns
        vx, vy ,ax, ay:
            doubles that contains the velocity and aceleration based on the given inputs
        """ 
        
        # z component of pendulum magnet
        z = -np.sqrt(self.l**2-x**2-y**2)+self.l+self.d       
        
        # Magnetic dipole vector m1 of pendulum magnet.     
        
        # Vector from Q to P
        rqp = np.array([x,y,z-self.l-self.d])
        
        # Magnetic dipole
        m1 = rqp/self.l
        
        # Finidng the sum of the magnetic forces
        Fb = np.array([0,0,0])
        B = np.array([0,0,0]) #magnetic field
        for i in range(self.NMAGNETS):
            #Squared distance between pendulum and table magnet
            r = (self.Mx[i]-x)**2 + (self.My[i]-y)**2 + z**2
            
            #Magnetic dipole vector of the table magnet
            m2 = self.mu_dir[i]
            
            #Unit vector of relative position (from m2 to m1)
            ur = np.array( [x-self.Mx[i], y-self.My[i], z] )/np.sqrt(r) 
            
            #Sum of the forces
            Fb = Fb + self.S[i] * ( (m1@m2)*ur + (m2@ur)*m1 + (m1@ur)*m2 - 5*(m1@ur)*(m2@ur)*ur ) / r**2
            
            #Constants for magnetic field
            Cm = -(4*np.pi*10**-7 * self.mu_magn[i]) / (4*np.pi)
            
            #Sum of mgnetic field
            B = B + Cm/r**(3/2) * (m2 - 3*(m2@ur)*ur)
        
        # Save the data of the net magnetic force and net magnetic field.
        self.FF1 = Fb
        self.BB1 = B
        
        # Finding tension
        
        # Unit vector from P to Q
        uT = -rqp/self.l 
        
        # Tension
        T = (self.m*9.8*(self.l+self.d-z)/self.l - Fb@uT)*uT
        
        # Acelerations
        ax = (Fb[0] - self.R*vx + T[0])/self.m
        ay = (Fb[1] - self.R*vy + T[1])/self.m
               
        return np.array([vx, vy, ax, ay])

    
    def next_value(self, x,y,vx,vy):
        """
        Calculates the next positions and velocities after a time lapse of self.h
        Find the 4 k-values of the Runge-Kutta method (RK4)
        @params
        x,y,vx,vy:
            doubles of the position and velocity of the pendulum magnet

        @returns
        xf,yf,vxf,vyf
            doubles that contains the final positions and velocities of the pendulum magnet
        """
        
        #K1 and calculations
        k1 = self.h*self.Fk(x,y,vx,vy)
        x1 = x + k1[0]/2
        y1 = y + k1[1]/2
        vx1 = vx + k1[2]/2
        vy1 = vy + k1[3]/2
        
        #Save magnetic Force and magnetic field
        self.FF2 = self.FF1
        self.BB2 = self.BB1

        #K2 and calculations
        k2 = self.h*self.Fk(x1,y1,vx1,vy1)
        x2 = x + k2[0]/2
        y2 = y + k2[1]/2
        vx2 = vx + k2[2]/2
        vy2 = vy + k2[3]/2

        #K3 and calculations
        k3 = self.h*self.Fk(x2,y2,vx2,vy2)
        x3 = x + k2[0]
        y3 = y + k2[1]
        vx3 = vx + k2[2]
        vy3 = vy + k2[3]

        #K4 and calculating final positions
        k4 = self.h*self.Fk(x3,y3,vx3,vy3)
        xf = x + 1/6*(k1[0] + 2*k2[0] + 2*k3[0] + k4[0])
        yf = y + 1/6*(k1[1] + 2*k2[1] + 2*k3[1] + k4[1])
        vxf = vx + 1/6*(k1[2] + 2*k2[2] + 2*k3[2] + k4[2])
        vyf = vy + 1/6*(k1[3] + 2*k2[3] + 2*k3[3] + k4[3])

        return np.array([xf, yf, vxf, vyf])
    
print('Done')

Done


### B. Animation

In [3]:
def animate_path(self):
    
    #local function for update each frame
    def update(j):
        
        # Clear the before plot
        ax.clear()
        
        # Plot the trajectories until the interval j
        ax.plot(self.X[:j],self.Y[:j],'-b',linewidth=0.5)
        ax.plot(self.X[j],self.Y[j],'o',markersize=5,color='b')
    
        # Plot the magnets
        for i in range(self.NMAGNETS):
            ax.plot(self.Mx[i],self.My[i],'or',markersize=10)
            ax.text(self.Mx[i]-0.02,self.My[i]-0.04,f'{np.round(self.mu_magn[i]*self.sign[i],2)}', fontsize=8)
        
        # Plot initial and final position
        ax.plot(self.X[0],self.Y[0],'Dg',label=f'Initial position ({np.round(self.X[0],3)},{np.round(self.Y[0],3)})', markersize=5)
        ax.plot(self.X[-1],self.Y[-1],'Dc',label='Final position', markersize=5)
        
        # Finding net magnetic Force
        FBx = self.FB[j][0]
        FBy = self.FB[j][1]
        FB_xy_mag = np.sqrt(FBx**2+FBy**2)
        
        # Plot net magnetic force vector
        ax.quiver(self.X[j],self.Y[j],0.25*FBx/FB_xy_mag,0.25*FBy/FB_xy_mag,color='red', label='Net Fb',angles='xy', scale_units='xy', scale=2)
        
        # Put on legend the Iteration and time of the frame
        ax.plot(0,0,'o',markersize=0, label=f'Iteration {j}')
        ax.plot(0,0,'o',markersize=0, label=f'Time {np.round(j*self.h,2)}s')
        
        # Put grids, legends, and limits
        plt.grid()
        plt.legend()
        plt.xlim(-self.lim,self.lim)
        plt.ylim(-self.lim,self.lim) 
    
    # Create figure and axis
    fig = plt.figure(figsize=(6,6))
    ax = fig.gca()
    
    # Put labels, title, grids, and legend
    plt.xlabel('X-axis [m]')
    plt.ylabel('Y-axis [m]')
    plt.title('Pendulum magnet trajectory')
    plt.grid()
    plt.legend()
    
    # Approximately Time per each plot
    f = 1/(0.25) 
    
    # Number of intervals that animation show per frame. Higher 'steps' lower the time of animation. steps < N
    steps=int(1/(self.h*f)) 
    
    # Number of intervals
    N = len(self.X)
    
    # Animate the movement
    self.ani=animation.FuncAnimation(fig,update,range(10,N,steps), repeat=False) 
    
    # Show animation
    plt.show()
print('Done')

Done


In [4]:
%matplotlib auto

Using matplotlib backend: TkAgg


## 2. Run the code and show results <a id='section2'></a>

### A. Normal run

In [6]:
# Cases of test
r = [0.1, 0.2, 0.28]    # Maximum distance of magnets to the center of coordinates
mu_P = 2                # magnetic dipole magnitude (pendulum)
mu = [1, 2.5, 6.25]     # Positive for atraction. Negative for repulsion.
dat = np.meshgrid(r, mu)
datr = dat[0].flatten()
datmu = dat[1].flatten()
CI = [[-0.14, -0.14,0,0], [-0.140-0.001, -0.140-0.001,0,0]]  #case 1

# Parameters
dath=[[0.00002,0.00002], [0.0005,0.0005], [0.0005,0.0005], [2e-6, 4e-7], 
      [0.0005,0.0005], [0.0005,0.0005], [0.0005,0.0005], [0.0005,0.0005], [0.0005,0.0005]]
NUM = len(r)*len(mu)*1*2 #Número de pruebas (quantity of r * quantity of mu * cases * 2)

# Create table magnets
Mx0 = [0.102, 0.06448, 0.05103, 0.009171, -0.001572, -0.04070, -0.05998, -0.126, -0.130]         #Positions table magnets
My0 = [-0.05954, 0.05833, -0.0001278, -0.08554, 0.08801, -0.004275, -0.09077, -0.04433, 0.08361]  
Mx0 = np.array(Mx0)
My0 = np.array(My0)

rho = np.sqrt(Mx0**2+My0**2)
Mx = Mx0/max(rho)
My = My0/max(rho)

# For convergence
def norm(A,B):
    s = [len(A[0]),len(B[0])]
    datx = [A[0], B[0]]
    daty = [A[1], B[1]]
    a = s.index(min(s))
    b = s.index(max(s))
    rel = s[b]/s[a]
    norm = [(datx[a][i]-datx[b][int(i*rel)])**2+(daty[a][i]-daty[b][int(i*rel)])**2 for i in range(s[a])]
    return sum(np.sqrt(norm))/s[a]

In [7]:
pends = [pendulum() for i in range(NUM)]
datPos= [[] for i in range(NUM)]

In [8]:
# Run code
count=6
for i in range(3,4):
    for j in range(2):
        Mxi = datr[i]*Mx
        Myi = datr[i]*My
        mui = [datmu[i]]*9
        CIj = CI[j]
        
        test=0
        h = dath[i][j]*5 #For start at 'dath[i][j]' predifined value on the next cycle
        for k in range(2):
            h=h/5
            N = int(4/h)
            pends[count].set_hN(h,N)
            pends[count].set_positions(CIj,Mxi,Myi)
            pends[count].set_magnetic_dipoles(mu_P,mui)
            inicio = time.time()
            X, Y, Vx, Vy = pends[count].find_path()
            datPos[count].append([X,Y,Vx,Vy])
            fin = time.time()
            print(f'Time count={count} ',fin-inicio,f'test={test} h={h}')  
            test +=1
        
        repetitions=2 #Count the number of repetitions to avoid exagerate repetitions
        while norm(datPos[count][-1][0:2], datPos[count][-2][0:2]) > 0.0003156: #For warranty convergence of path
            h = h/5
            N = int(4/h)
            pends[count].set_hN(h,N)
            pends[count].set_positions(CIj,Mxi,Myi)
            pends[count].set_magnetic_dipoles(mu_P,mui)
            inicio = time.time()
            X, Y, Vx, Vy = pends[count].find_path()
            datPos[count].append([X,Y,Vx,Vy])
            fin = time.time()
            print(f'Time count={count} ',fin-inicio,f'test={test} h={h}')  
            test +=1
            repetitions+=1
            if repetitions>5:
                print('Breaking point')
                break
        dath[i][j] = h #For save value
        if j==0: #For assigment the h value to the second initial condition
            dath[i][1] = h*5
        print() #For see the data in a separate way
        count += 1 #For passing to the next case
    start=0

Time count=6  2258.212324619293 test=0 h=2e-06


KeyboardInterrupt: 

In [9]:
selfs = [pends[i].get_self() for i in range(6,7)]

In [11]:
for i in range(0,1):
    X = selfs[i].X
    Y = selfs[i].Y
    Vx = selfs[i].Vx
    Vy = selfs[i].Vy
    df = pd.DataFrame({'X':X, 'Y':Y, 'Vx':Vx})
    df.to_csv(f'data_path_{i}.csv')
    FB = selfs[i].FB
    B = selfs[i].B
    df2 = pd.DataFrame({'FB':FB, 'B':B})
    df2.to_csv(f'data_forces_{i}.csv')

### B. Load data of other runs

In [7]:
for i in range(18):
    datain = pd.read_csv(f'paths/data_path_{i}.csv', index_col=0) #For input path
    X = datain['X'].values
    Y = datain['Y'].values
    Vx = datain['Vx'].values
    Vy = datain['Vy'].values
    pends[i].input_path([X,Y,Vx,Vy])
    datain2 = pd.read_csv('paths/parameters.csv',index_col=0) #For input conditions
    h,N = datain2['h'].values[i], datain2['N'].values[i]
    pends[i].set_hN(h,N)
    CIi = CI[i%2]
    Mxi = datr[i//2]*Mx
    Myi = datr[i//2]*My
    mui = [datmu[i//2]]*9
    pends[i].set_positions(CIi,Mxi,Myi)
    pends[i].set_magnetic_dipoles(mu_P,mui)
print('Done')
selfs = [pends[i].get_self() for i in range(NUM)] #Get selfs

Done


### C. Plot potential and trajectory

In [8]:
# Plot the potential 
dat = [[0, 6, 12], [2, 8, 14], [4, 10, 16]] #For plot some particular cases
a = [0.2, 0.2, 0.3] #For setting limits to the plot
for i in range(3):
    for j in range(3):
        k=dat[i][j]
        pends[k].plot_potential(res=100, a=a[i]) #res=resolution (optional), a=limits of plot (optional)
        #plt.savefig(f'Apos_potential_r={r[i]}_mu={mu[j]}.png',dpi=250)

In [ ]:
# Animate the trajectory (only for normal runs)
k=0
pends[k].set_lim(0.25) #Adjust the xlims and ylims of the plot. Increase to see small the set up.
animate_path(selfs[k])

In [ ]:
# Plot mechanical energy (only for normal runs)
for k in [0, 2, 8, 10, 14, 16]: #For some particular cases
    # Pendulum constants
    m = selfs[k].m
    d = selfs[k].d
    l = selfs[k].l
    # Positions
    X = selfs[k].X
    Y= selfs[k].Y
    Z = -np.sqrt(selfs[k].l**2 - X**2 - Y**2) +l+d
    # Velocities
    Vx = selfs[k].Vx
    Vy = selfs[k].Vy
    Vz = -(X*Vx+Y*Vy)/(Z-l-d)
    # Mgnetic dipoles constants and vector
    m1 = selfs[k].mu_P_magn
    Um1_x = X/l                 #X component  
    Um1_y = Y/l                 #Y component 
    Um1_z = (Z-l-d)/l           #Z component
    NINTERVALS = len(selfs[k].B) # Number of intervals
    # Magnetic Fields
    Bx = np.array([selfs[k].B[i][0] for i in range(NINTERVALS)])
    By = np.array([selfs[k].B[i][1] for i in range(NINTERVALS)])
    Bz = np.array([selfs[k].B[i][2] for i in range(NINTERVALS)])
    # Potentials
    Ug = m*9.8*(Z-d)
    Um = -m1 * (Um1_x*Bx + Um1_y*By + Um1_z*Bz)
    U = Ug+Um-min(Ug+Um)
    # Kinetic Energy
    K = 0.5*m*(Vx**2+Vy**2+Vz**2)
    # Plot Mehanical Energy
    plt.figure()
    t = np.linspace(0, NINTERVALS*selfs[k].h, NINTERVALS)
    plt.plot(t,U+K,'-r',label = 'Mechanical Energy',linewidth=0.4)
    #plt.plot(t,K,'-b',label = 'Kinetic Energy',linewidth=0.5)
    #plt.plot(t,U,color='orange',label = 'Potential Energy',linewidth=0.5)
    plt.ylabel('Mechanical Energy [J]', fontsize = 12)
    plt.xlabel('Time [s]', fontsize = 12)
    plt.title('Mechanical Energy of the system', fontsize = 15)
    plt.legend(fontsize=10, ncol=1)
    plt.xticks(size = 10)
    plt.yticks(size = 10)
    plt.savefig(f'Apos_mechanical_energy_{k+1}_r={datr[k//2]}_mu={datmu[k//2]}_CI_case_{k%2 + 1}.png',dpi=300)
    plt.show()

In [109]:
# Plot magnets locations
pends[0].plot_table()
Mx = selfs[0].Mx
My = selfs[0].My

# Plot magnets
for i in range(len(Mx)):
    plt.plot(Mx[i],My[i],'or')
    plt.text(Mx[i]-0.01,My[i]-0.02,f'{i}')
plt.text(0.05,0.14,r' $\mu =$'+str(mu[2])+r'[$Am^2$]')
# Put limits, labels, legend and title
lim = selfs[0].Table_SIZE/2 + 0.04
plt.xlim(-lim,lim)
plt.ylim(-lim,lim)
plt.xlabel('X-axis [m]')
plt.ylabel('Y-axis [m]')
plt.legend(loc='upper right')
plt.title('Table Magnets')
plt.show()

In [24]:
# Plot the table, trajectory, and vector field.
for i in range(12,16):
    # Plot table and path
    pends[i].plot_table()              
    pends[i].plot_path()

    # Plot vector space
    pends[i].plot_vector_space(res=30) #res: resolution. Increase to obtain more vectors.
    plt.legend(loc='upper left',fontsize=10)
    plt.savefig(f'r={datr[i//2]} mu={datmu[i//2]}/Apos_vector_space_{i+1}_r={datr[i//2]}_mu={datmu[i//2]}.png',dpi=300)
    plt.show()

### D. Aditional isues

#### I. Save data (if needed)

In [53]:
selfs = [pends[i].get_self() for i in range(14)]

In [56]:
h=[]
N=[]
for i in range(14):
    h.append(selfs[i].h)
    N.append(selfs[i].N)
df = pd.DataFrame({'h':h, 'N':N})
df.to_csv(f'paths/parameters.csv')

In [52]:
X = datPos[13][-1][0]
Y = datPos[13][-1][1]
Vx = datPos[13][-1][2]
Vy = datPos[13][-1][3]
df = pd.DataFrame({'X':X, 'Y':Y, 'Vx':Vx, 'Vy':Vy})
df.to_csv(f'paths/data_path_{i}.csv')

#### II. Read data (if needed)

In [44]:
datain = pd.read_csv('paths/data_path_13.csv', index_col=0)
X1 = datain['X'].values
Y1 = datain['Y'].values

#### IV. Parameters founded of convergence

0.05673275468615218 
Time  0.1606311798095703
Trajectory that doesn't converge:

0.00031563293988951034
Time  0.16422510147094727
Trajectory that converge:

## 3. Analyse the data <a id='section3'></a>

In [9]:
Lambdas = [[] for i in range(9)]

### A. Phase Portrait

In [26]:
#Phase portrait 3D (x,y,v)

for i in range(14,16):
    # Get positions and velocity
    X=selfs[i].X
    Y=selfs[i].Y
    Vx=selfs[i].Vx
    Vy=selfs[i].Vy
    V = np.sqrt(Vx**2+Vy**2)*np.sign(Vy) #With sign

    # Create projection 3d
    plt.figure()
    ax = plt.axes(projection='3d')

    # Plot phae portrait
    plt.plot(X,Y,V,'-r',linewidth=0.5)

    # Put tilte and labels
    plt.title(f'Phase portrait path {i+1}', fontsize=15)
    plt.xlabel(r'X [m]',fontsize=12)
    plt.ylabel(r'Y [m]',fontsize=12)
    ax.set_zlabel(r'V sign(Vx) [m/s]',fontsize=12)
    plt.xticks(fontsize=9)
    plt.yticks(fontsize=9)
    for t in ax.zaxis.get_major_ticks(): t.label.set_fontsize(9)
    plt.savefig(f'r={datr[i//2]} mu={datmu[i//2]}/Apos_phase_portrait_{i+1}.png',dpi=300)
    plt.show()

    # Time series
    plt.figure()

    # Create time array
    t=np.linspace(0,selfs[i].h*selfs[i].N,len(X))

    # Plot time series
    plt.plot(t,X,'-b',linewidth=0.5)

    # Put the title and labels
    plt.title(f'Time series of X path {i+1}',fontsize=15)
    plt.xlabel('Time [s]',fontsize=12)
    plt.ylabel('X [m]',fontsize=12)
    plt.savefig(f'r={datr[i//2]} mu={datmu[i//2]}/Apos_time_series_of_X_{i+1}.png',dpi=300)
    plt.show()

In [10]:
# Superposition
plt.figure()
ax = plt.axes(projection='3d')
colors=['blue','red']*(NUM//2)
#colors=['blue','red','green','orange']*3
for i in range(16,18):
    # Get positions and velocity
    X=selfs[i].X
    Y=selfs[i].Y
    Vx=selfs[i].Vx
    Vy=selfs[i].Vy
    V = np.sqrt(Vx**2+Vy**2)*np.sign(Vy) #With sign
    
    # Plot phae portrait
    plt.plot(X,Y,V,color=colors[i],linewidth=0.8,label=f'Path {i+1}',alpha=0.9)

# Put tilte and labels
plt.title('Phase portrait 3D', fontsize=15)
plt.xlabel(r'X [m]',fontsize=12)
plt.ylabel(r'Y [m]',fontsize=12)
plt.legend()
ax.set_zlabel(r'V sign(Vx) [m/s]',fontsize=12)
plt.xticks(fontsize=9)
plt.yticks(fontsize=9)
for t in ax.zaxis.get_major_ticks(): t.label.set_fontsize(9)
plt.show()
plt.savefig(f'r={datr[i//2]} mu={datmu[i//2]}/Apos_phase_portrait_{i}{i+1}.png',dpi=300)

### B. Difference of paths and Liapunov exponent

In [35]:
# Differents between paths
pair=3 #From 0 to 8

X1, Y1 = selfs[pair*2].X, selfs[pair*2].Y 
X2, Y2 = selfs[pair*2+1].X, selfs[pair*2+1].Y

# Distance between paths
nn = min([len(X1),len(X2)])
D=np.sqrt((X1[:nn]-X2[:nn])**2 + ((Y1[:nn]-Y2[:nn])**2))

# Time array
t=np.linspace(0,selfs[0].h * selfs[0].N ,len(D))

plt.figure()

# Plot difference vs time
#plt.plot(t,D, label='Distance')
plt.plot(t, np.log(D/D[0]), '-b',label='Log distance')

# Put labels, title, legend, ticks and grid
plt.ylabel(r'Log(D(t)/D(0)) [dimensionless]', fontsize = 12)
plt.xlabel('Time [s]', fontsize = 12)
plt.title('Chaotic behaviour, changing 1mm the initial positions', fontsize = 15)
plt.legend(fontsize=10, ncol=1)
plt.xticks(size = 10)
plt.yticks(size = 10)
plt.grid(True)

# Save image
plt.savefig(f'r={datr[pair]} mu={datmu[pair]}/Apos_diference_trajectories_{pair*2+1}{pair*2+2}.png',dpi=300)

# Show image
plt.show()

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_12512\195123931.py:18: RuntimeWarning: divide by zero encountered in true_divide
  plt.plot(t, np.log(D/D[0]), '-b',label='Log distance')
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_12512\195123931.py:18: RuntimeWarning: invalid value encountered in true_divide
  plt.plot(t, np.log(D/D[0]), '-b',label='Log distance')


In [153]:
#Check start and end
ti = 0
tf = 0.7263
start = int(ti/selfs[pair*2].h)
end = int(tf/selfs[pair*2].h)
print(start, end)

0 7262


In [156]:
starts = [108999, 0, 0, 0, 0, 0, 0, 0]
ends =   [561249, 7117, 7262, 0, 0, 0, 0, 0]

start = starts[pair]
end = ends[pair]

def linear(x, m, b):
    return x*m + b

t_set = t[start:end+1]
D_set = D[start:end+1]
params, cov = curve_fit(linear, t_set, np.log(D_set/D_set[0]))
var = np.sqrt(np.diag(cov))

# Plot difference vs time
plt.figure()
plt.plot(t_set, np.log(D_set/D_set[0]),color='blue', label='Log distance')
plt.plot(t_set, linear(t_set, params[0], params[1]), color='orange', label=f'Linear Fit: log(D)={np.round(params[0],2)}t+{np.round(params[1],2)}')

# Put labels, title, legend, ticks and grid
plt.ylabel('Log(D(t) / D0) [dimensionless]', fontsize = 12)
plt.xlabel('Time [s]', fontsize = 12)
plt.title('Chaotic behaviour, changing 1mm initial positions', fontsize = 15)
plt.legend(fontsize=10, ncol=1)
plt.xticks(size = 10)
plt.yticks(size = 10)

xtext = 5/7*(t_set[-1]-t_set[0]) + t_set[0]
log_set = np.log(D_set/D_set[0])
ytext = 2/6*(max(log_set)-min(log_set)) + min(log_set)

plt.text(xtext, ytext, f'lambda = {np.round(params[0],3)}', fontsize=10)
plt.grid(True)
plt.savefig(f'r={datr[pair]} mu={datmu[pair]}/Apos_Lyapunov_{2*pair+1}{2*pair+2}.png',dpi=300)
plt.show()
Lambdas[pair] = params[0]

In [157]:
plt.figure()
t_Ly = np.linspace(t[start], t[end-1300],200)
y_Ly = D[start]*np.e**(params[0]*(t_Ly)+params[1])
plt.plot(t_Ly, y_Ly, '-',label=r'Fit $D(t) = %.5f \ e^{%.2f t + %.2f}$'%(np.round(D[start],5),np.round(params[0],2),np.round(params[1],2))
         ,linewidth=1.5, color='red', alpha=0.6)

#start2 = 121
#end2 = 166
#t_Ly2 = np.linspace(t[start2], t[end2+2],200)
#y_Ly2 = D[start2]*np.e**(params2[0]*(t_Ly2)+params2[1])
#plt.plot(t_Ly2, y_Ly2, '-',label=f'Fit 2 D(t) = {np.round(D[start2],4)}e^({np.round(params2[0],2)} t - {-np.round(params2[1],2)})',linewidth=1.5, color='orange', alpha=0.6)
plt.plot(t,D,'-b',label='Distance', linewidth=1.5, alpha=0.6)

plt.ylabel('D(t) [m]', fontsize = 12)
plt.xlabel('Time [s]', fontsize = 12)
plt.ylim(-0.0055, 0.42)
plt.title('Chaotic behaviour, changing 1mm initial positions', fontsize = 15)
plt.legend(fontsize=10, ncol=1, loc='upper left')
plt.xticks(size = 10)
plt.yticks(size = 10)
plt.savefig(f'r={datr[pair]} mu={datmu[pair]}/Apos_Exponential_divergence_{2*pair+1}{2*pair+2}.png',dpi=300)
plt.show()

In [19]:
Lambdas

NameError: name 'Lambdas' is not defined

### C. Lyapunov Exponent Analysis

In [25]:
L = np.array([[1.2, 2.7, 3.2],[2.7, 1.4, 3],[3.4, 1.1, 2.7]])

In [34]:
plt.figure()
plt.title(r"$\lambda$ parameter space",fontsize=14)
plt.xlabel(r'Magnetic dipole magnitudes [$Am^2$]',fontsize=12)
plt.ylabel(r'Radius delimiter zone [$m$]',fontsize=12)
plt.xticks(range(3),[str(mu[i]) for i in range(3)])
plt.yticks(range(3),[str(r[i]) for i in range(3)])
plt.imshow(L)
cbar = plt.colorbar()
cbar.set_label('$\lambda \ [1/s]$', rotation=90, fontsize=12)
plt.savefig('MAIN SUMMARY RESULT Lambda parameters space.png',dpi=250)
plt.show()